# Model performance

In [3]:
import numpy as np
import shap
import pickle
from datasets import load_dataset
from src.dataset_info import get_dataset_info
from transformers import pipeline, AutoTokenizer
import pandas as pd
from datasets import load_dataset, Dataset
import os
from tqdm import tqdm
from src.utils import token_segments, text_ft_index_ends

# from src.models import Model
import lightgbm as lgb
from src.models import WeightedEnsemble, StackModel, AllAsTextModel
from src.joint_masker import JointMasker
import argparse
import scipy as sp
from sklearn.metrics import roc_auc_score

In [14]:
def run_model(model_type, ds_type, test_set_size=100, tab_scale_factor=2):
    di = get_dataset_info(ds_type, model_type)
    # Data
    train_df = load_dataset(
        di.ds_name, split="train", download_mode="force_redownload"
    ).to_pandas()
    y_train = train_df[di.label_col]

    test_df = load_dataset(
        di.ds_name, split="test", download_mode="force_redownload"
    ).to_pandas()
    test_df_sample = test_df.sample(test_set_size, random_state=55)

    # Models
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
    if model_type == "all_text":
        text_pipeline = pipeline(
            "text-classification",
            # model=di.text_model_name,
            model="../models/wine/glowing-morning-9/checkpoint-6705",
            tokenizer=tokenizer,
            device="cuda:0",
            truncation=True,
            padding=True,
            top_k=None,
        )
        # Define how to convert all columns to a single string

        def cols_to_str_fn(array):
            return " | ".join(
                [f"{col}: {val}" for col, val in zip(
                    di.tab_cols + di.text_cols, array)]
            )

        model = AllAsTextModel(
            text_pipeline=text_pipeline,
            cols_to_str_fn=cols_to_str_fn,
            # cols=di.tab_cols + di.text_cols
        )
    else:
        text_pipeline = pipeline(
            "text-classification",
            model=di.text_model_name,
            tokenizer=tokenizer,
            device="cuda:0",
            truncation=True,
            padding=True,
            top_k=None,
        )
        # Define how to convert the text columns to a single string
        if len(di.text_cols) == 1:

            def cols_to_str_fn(array):
                return array[0]

        else:

            def cols_to_str_fn(array):
                return " | ".join(
                    [f"{col}: {val}" for col, val in zip(di.text_cols, array)]
                )

        # LightGBM requires explicitly marking categorical features
        train_df[di.categorical_cols] = train_df[di.categorical_cols].astype(
            "category")
        test_df_sample[di.categorical_cols] = test_df_sample[
            di.categorical_cols
        ].astype("category")

        tab_model = lgb.LGBMClassifier(random_state=42)
        tab_model.fit(train_df[di.tab_cols], y_train)

        if model_type == "ensemble_50":
            model = WeightedEnsemble(
                tab_model=tab_model,
                text_pipeline=text_pipeline,
                text_weight=0.5,
                cols_to_str_fn=cols_to_str_fn,
            )
        elif model_type == "ensemble_75":
            model = WeightedEnsemble(
                tab_model=tab_model,
                text_pipeline=text_pipeline,
                text_weight=0.75,
                cols_to_str_fn=cols_to_str_fn,
            )
        elif model_type == "ensemble_25":
            model = WeightedEnsemble(
                tab_model=tab_model,
                text_pipeline=text_pipeline,
                text_weight=0.25,
                cols_to_str_fn=cols_to_str_fn,
            )
        elif model_type == "stack":
            """
            For the stack model, we make predictions on the validation set. These predictions
            are then used as features for the stack model (another LightGBM model) along with
            the other tabular features. In doing so the stack model learns, depending on the
            tabular features, when to trust the tabular model and when to trust the text model.
            """
            val_df = load_dataset(
                di.ds_name, split="validation", download_mode="force_redownload"
            ).to_pandas()
            val_df[di.categorical_cols] = val_df[di.categorical_cols].astype(
                "category")
            y_val = val_df[di.label_col]
            val_text = list(map(cols_to_str_fn, val_df[di.text_cols].values))

            # Training set is the preditions from the tabular and text models on the validation set
            # plus the tabular features from the validation set
            text_val_preds = text_pipeline(val_text)
            # text_val_preds = np.array(
            #     [format_text_pred(pred) for pred in text_val_preds]
            # )
            text_val_preds = np.array(
                [[lab["score"] for lab in pred] for pred in text_val_preds]
            )

            # add text and tabular predictions to the val_df
            stack_val_df = val_df[di.tab_cols]
            tab_val_preds = tab_model.predict_proba(stack_val_df)
            for i in range(text_val_preds.shape[1]):
                stack_val_df[f"text_pred_{i}"] = text_val_preds[:, i]
            for i in range(tab_val_preds.shape[1]):
                stack_val_df[f"tab_pred_{i}"] = tab_val_preds[:, i]

            stack_model = lgb.LGBMClassifier(random_state=42)
            stack_model.fit(stack_val_df, y_val)

            model = StackModel(
                tab_model=tab_model,
                text_pipeline=text_pipeline,
                stack_model=stack_model,
                cols_to_str_fn=cols_to_str_fn,
            )
        else:
            raise ValueError(f"Invalid model type of {model_type}")

    np.random.seed(1)
    test_sample_vals = test_df_sample[di.tab_cols + di.text_cols].values
    test_vals = test_df[di.tab_cols + di.text_cols].values

    return (
        model.predict(test_sample_vals),
        test_df_sample[di.label_col].values,
        model.predict(test_vals),
        test_df[di.label_col].values,
    )

## Fake Job Postings (ROC-AUC)

In [24]:
ds_type = "fake"

for model_type in ["ensemble_25", "ensemble_50", "ensemble_75", "all_text", "stack"]:
    sample_preds, sample_y, preds, y = run_model(
        model_type, ds_type=ds_type, tab_scale_factor=1
    )

    print(f"\n{model_type} on {ds_type}")
    print(f"Test accuaracy: {np.mean(np.argmax(preds, axis=1) == y)}")
    print(
        f"Test accuaracy (sample): {np.mean(np.argmax(sample_preds, axis=1) == sample_y)}"
    )
    print(f"Test AUC: {roc_auc_score(y, preds[:, 1])}")
    print(f"Test AUC (sample): {roc_auc_score(sample_y, sample_preds[:, 1])}")
    print(f"Test % by label: {[np.mean(y == label) for label in np.unique(y)]}")
    print(
        f"Test % by label (sample): {[np.mean(sample_y == label) for label in np.unique(y)]}"
    )

Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--fake_job_postings2_ordinal-d873cc356e36f3d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--fake_job_postings2_ordinal-d873cc356e36f3d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100it [00:00, 90825.12it/s]
3182it [00:00, 54504.85it/s]



ensemble_25 on fake
Test accuaracy: 0.9387177875549969
Test accuaracy (sample): 0.93
Test AUC: 0.8744072444723762
Test AUC (sample): 0.808421052631579
Test % by label: [0.9566310496543055, 0.04336895034569453]
Test % by label (sample): [0.95, 0.05]


Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--fake_job_postings2_ordinal-d873cc356e36f3d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--fake_job_postings2_ordinal-d873cc356e36f3d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100it [00:00, 97792.12it/s]
3182it [00:00, 53235.83it/s]



ensemble_50 on fake
Test accuaracy: 0.9619736015084852
Test accuaracy (sample): 0.96
Test AUC: 0.9189305642842179
Test AUC (sample): 0.9136842105263159
Test % by label: [0.9566310496543055, 0.04336895034569453]
Test % by label (sample): [0.95, 0.05]


Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--fake_job_postings2_ordinal-d873cc356e36f3d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--fake_job_postings2_ordinal-d873cc356e36f3d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100it [00:00, 95847.90it/s]
3182it [00:00, 53839.94it/s]


...

Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--fake_job_postings2_ordinal-d873cc356e36f3d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--fake_job_postings2_ordinal-d873cc356e36f3d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 2994.51it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--fake_job_postings2_ordinal-d873cc356e36f3d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100it [00:00, 95108.93it/s]
3182it [00:00, 54891.09it/s]



stack on fake
Test accuaracy: 0.943117536140792
Test accuaracy (sample): 0.93
Test AUC: 0.9087501666380999
Test AUC (sample): 0.9557894736842105
Test % by label: [0.9566310496543055, 0.04336895034569453]
Test % by label (sample): [0.95, 0.05]


## IMDB  (ROC-AUC)

In [25]:
ds_type = "imdb_genre"

for model_type in ["ensemble_25", "ensemble_50", "ensemble_75", "all_text", "stack"]:
    sample_preds, sample_y, preds, y = run_model(
        model_type, ds_type=ds_type, tab_scale_factor=1
    )

    print(f"\n{model_type} on {ds_type}")
    print(f"Test accuaracy: {np.mean(np.argmax(preds, axis=1) == y)}")
    print(
        f"Test accuaracy (sample): {np.mean(np.argmax(sample_preds, axis=1) == sample_y)}"
    )
    print(f"Test AUC: {roc_auc_score(y, preds[:, 1])}")
    print(f"Test AUC (sample): {roc_auc_score(sample_y, sample_preds[:, 1])}")
    print(f"Test % by label: {[np.mean(y == label) for label in np.unique(y)]}")
    print(
        f"Test % by label (sample): {[np.mean(sample_y == label) for label in np.unique(y)]}"
    )

Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--imdb_genre_prediction_ordinal-95c476e18d2d7064/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--imdb_genre_prediction_ordinal-95c476e18d2d7064/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100it [00:00, 688719.87it/s]
200it [00:00, 964207.82it/s]



ensemble_25 on imdb_genre
Test accuaracy: 0.78
Test accuaracy (sample): 0.79
Test AUC: 0.8530677609848865
Test AUC (sample): 0.8262626262626263
Test % by label: [0.485, 0.515]
Test % by label (sample): [0.55, 0.45]


Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--imdb_genre_prediction_ordinal-95c476e18d2d7064/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--imdb_genre_prediction_ordinal-95c476e18d2d7064/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100it [00:00, 759837.68it/s]
200it [00:00, 975419.53it/s]



ensemble_50 on imdb_genre
Test accuaracy: 0.785
Test accuaracy (sample): 0.79
Test AUC: 0.8646782103893504
Test AUC (sample): 0.8513131313131314
Test % by label: [0.485, 0.515]
Test % by label (sample): [0.55, 0.45]


Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--imdb_genre_prediction_ordinal-95c476e18d2d7064/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--imdb_genre_prediction_ordinal-95c476e18d2d7064/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100it [00:00, 649273.07it/s]
200it [00:00, 969781.27it/s]


...

Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--imdb_genre_prediction_ordinal-95c476e18d2d7064/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--imdb_genre_prediction_ordinal-95c476e18d2d7064/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 2481.35it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--imdb_genre_prediction_ordinal-95c476e18d2d7064/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/


stack on imdb_genre
Test accuaracy: 0.74
Test accuaracy (sample): 0.73
Test AUC: 0.8106295666099489
Test AUC (sample): 0.7846464646464647
Test % by label: [0.485, 0.515]
Test % by label (sample): [0.55, 0.45]


## Kickstarter  (ROC-AUC)

In [26]:
ds_type = "kick"

for model_type in ["ensemble_25", "ensemble_50", "ensemble_75", "all_text", "stack"]:
    sample_preds, sample_y, preds, y = run_model(
        model_type, ds_type=ds_type, tab_scale_factor=1
    )

    print(f"\n{model_type} on {ds_type}")
    print(f"Test accuaracy: {np.mean(np.argmax(preds, axis=1) == y)}")
    print(
        f"Test accuaracy (sample): {np.mean(np.argmax(sample_preds, axis=1) == sample_y)}"
    )
    print(f"Test AUC: {roc_auc_score(y, preds[:, 1])}")
    print(f"Test AUC (sample): {roc_auc_score(sample_y, sample_preds[:, 1])}")
    print(f"Test % by label: {[np.mean(y == label) for label in np.unique(y)]}")
    print(
        f"Test % by label (sample): {[np.mean(sample_y == label) for label in np.unique(y)]}"
    )

Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 2821.28it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--kick_starter_funding_ordinal-bc45dae77b1c676d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--kick_starter_funding_ordinal-bc45dae77b1c676d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100it [00:00, 341834.07it/s]
21626it [00:00, 332713.50it/s]



ensemble_25 on kick
Test accuaracy: 0.6999445112364746
Test accuaracy (sample): 0.66
Test AUC: 0.7410626481173135
Test AUC (sample): 0.7507507507507508
Test % by label: [0.680384722093776, 0.319615277906224]
Test % by label (sample): [0.63, 0.37]


Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--kick_starter_funding_ordinal-bc45dae77b1c676d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--kick_starter_funding_ordinal-bc45dae77b1c676d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100it [00:00, 437818.79it/s]
21626it [00:00, 329554.85it/s]


...

Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--kick_starter_funding_ordinal-bc45dae77b1c676d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--kick_starter_funding_ordinal-bc45dae77b1c676d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 2820.02it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--kick_starter_funding_ordinal-bc45dae77b1c676d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100it [00:00, 416928.83it/s]
21626it [00:00, 326971.04it/s]



stack on kick
Test accuaracy: 0.7072967724035882
Test accuaracy (sample): 0.65
Test AUC: 0.7491010899483486
Test AUC (sample): 0.7211497211497212
Test % by label: [0.680384722093776, 0.319615277906224]
Test % by label (sample): [0.63, 0.37]


## Jigsaw (ROC-AUC)

In [27]:
ds_type = "jigsaw"

for model_type in ["ensemble_25", "ensemble_50", "ensemble_75", "all_text", "stack"]:
    sample_preds, sample_y, preds, y = run_model(
        model_type, ds_type=ds_type, tab_scale_factor=1
    )

    print(f"\n{model_type} on {ds_type}")
    print(f"Test accuaracy: {np.mean(np.argmax(preds, axis=1) == y)}")
    print(
        f"Test accuaracy (sample): {np.mean(np.argmax(sample_preds, axis=1) == sample_y)}"
    )
    print(f"Test AUC: {roc_auc_score(y, preds[:, 1])}")
    print(f"Test AUC (sample): {roc_auc_score(sample_y, sample_preds[:, 1])}")
    print(f"Test % by label: {[np.mean(y == label) for label in np.unique(y)]}")
    print(
        f"Test % by label (sample): {[np.mean(sample_y == label) for label in np.unique(y)]}"
    )

Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--jigsaw_unintended_bias100K_ordinal-8e97391c4f489562/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--jigsaw_unintended_bias100K_ordinal-8e97391c4f489562/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100it [00:00, 437818.79it/s]
25000it [00:00, 179864.52it/s]



ensemble_25 on jigsaw
Test accuaracy: 0.94256
Test accuaracy (sample): 0.95
Test AUC: 0.9321638495642937
Test AUC (sample): 0.8378947368421054
Test % by label: [0.9426, 0.0574]
Test % by label (sample): [0.95, 0.05]


Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--jigsaw_unintended_bias100K_ordinal-8e97391c4f489562/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--jigsaw_unintended_bias100K_ordinal-8e97391c4f489562/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100it [00:00, 438276.28it/s]
25000it [00:00, 459371.86it/s]



ensemble_50 on jigsaw
Test accuaracy: 0.9448
Test accuaracy (sample): 0.95
Test AUC: 0.9456286008527085
Test AUC (sample): 0.8631578947368422
Test % by label: [0.9426, 0.0574]
Test % by label (sample): [0.95, 0.05]


Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--jigsaw_unintended_bias100K_ordinal-8e97391c4f489562/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--jigsaw_unintended_bias100K_ordinal-8e97391c4f489562/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100it [00:00, 422812.90it/s]
25000it [00:00, 175630.87it/s]


...

Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--jigsaw_unintended_bias100K_ordinal-8e97391c4f489562/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--jigsaw_unintended_bias100K_ordinal-8e97391c4f489562/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 3004.52it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--jigsaw_unintended_bias100K_ordinal-8e97391c4f489562/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100it [00:00, 353651.26it/s]
25000it [00:00, 178293.77it/s]



stack on jigsaw
Test accuaracy: 0.93208
Test accuaracy (sample): 0.95
Test AUC: 0.9158202495728693
Test AUC (sample): 0.9515789473684211
Test % by label: [0.9426, 0.0574]
Test % by label (sample): [0.95, 0.05]


## Product Sentiment

In [29]:
ds_type = "prod_sent"

for model_type in ["ensemble_25", "ensemble_50", "ensemble_75", "all_text", "stack"]:
    sample_preds, sample_y, preds, y = run_model(
        model_type, ds_type=ds_type, tab_scale_factor=1
    )

    print(f"\n{model_type} on {ds_type}")
    print(f"Test accuaracy: {np.mean(np.argmax(preds, axis=1) == y)}")
    print(
        f"Test accuaracy (sample): {np.mean(np.argmax(sample_preds, axis=1) == sample_y)}"
    )
    # print(f"Test AUC: {roc_auc_score(y, preds[:, 1])}")
    # print(f"Test AUC (sample): {roc_auc_score(sample_y, sample_preds[:, 1])}")
    print(f"Test % by label: {[np.mean(y == label) for label in np.unique(y)]}")
    print(
        f"Test % by label (sample): {[np.mean(sample_y == label) for label in np.unique(y)]}"
    )

Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--product_sentiment_machine_hack_ordinal-bbe910d41184c5e7/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--product_sentiment_machine_hack_ordinal-bbe910d41184c5e7/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100it [00:00, 719434.65it/s]
1273it [00:00, 1353790.31it/s]



ensemble_25 on prod_sent
Test accuaracy: 0.8908091123330715
Test accuaracy (sample): 0.95
Test % by label: [0.010997643362136685, 0.0589159465828751, 0.5875883739198743, 0.3424980361351139]
Test % by label (sample): [0.01, 0.02, 0.62, 0.35]


Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--product_sentiment_machine_hack_ordinal-bbe910d41184c5e7/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--product_sentiment_machine_hack_ordinal-bbe910d41184c5e7/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100it [00:00, 719434.65it/s]
1273it [00:00, 1319009.14it/s]



ensemble_50 on prod_sent
Test accuaracy: 0.8695993715632364
Test accuaracy (sample): 0.9
Test % by label: [0.010997643362136685, 0.0589159465828751, 0.5875883739198743, 0.3424980361351139]
Test % by label (sample): [0.01, 0.02, 0.62, 0.35]


Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--product_sentiment_machine_hack_ordinal-bbe910d41184c5e7/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--product_sentiment_machine_hack_ordinal-bbe910d41184c5e7/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100it [00:00, 932067.56it/s]
1273it [00:00, 1058554.52it/s]


...

Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--product_sentiment_machine_hack_ordinal-bbe910d41184c5e7/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--product_sentiment_machine_hack_ordinal-bbe910d41184c5e7/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 3104.59it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--product_sentiment_machine_hack_ordinal-bbe910d41184c5e7/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100it [00:00, 746317.44it/s]
1273it [00:00, 1028181.97it/s]



stack on prod_sent
Test accuaracy: 0.6496465043205027
Test accuaracy (sample): 0.68
Test % by label: [0.010997643362136685, 0.0589159465828751, 0.5875883739198743, 0.3424980361351139]
Test % by label (sample): [0.01, 0.02, 0.62, 0.35]


## Wine

In [31]:
def run_model(model_type, ds_type, test_set_size=100, tab_scale_factor=2):
    di = get_dataset_info(ds_type, model_type)
    # Data
    train_df = load_dataset(
        di.ds_name,
        split="train",  # download_mode="force_redownload"
    ).to_pandas()
    y_train = train_df[di.label_col]

    test_df = load_dataset(
        di.ds_name,
        split="test",  # download_mode="force_redownload"
    ).to_pandas()
    test_df_sample = test_df.sample(test_set_size, random_state=55)

    # Models
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
    if model_type == "all_text":
        text_pipeline = pipeline(
            "text-classification",
            # model=di.text_model_name,
            model="../models/wine/glowing-morning-9/checkpoint-6705",
            tokenizer=tokenizer,
            device="cuda:0",
            truncation=True,
            padding=True,
            top_k=None,
        )
        # Define how to convert all columns to a single string

        def cols_to_str_fn(array):
            return " | ".join(
                [f"{col}: {val}" for col, val in zip(di.tab_cols + di.text_cols, array)]
            )

        model = AllAsTextModel(
            text_pipeline=text_pipeline,
            cols_to_str_fn=cols_to_str_fn,
            # cols=di.tab_cols + di.text_cols
        )
    else:
        text_pipeline = pipeline(
            "text-classification",
            model=di.text_model_name,
            tokenizer=tokenizer,
            device="cuda:0",
            truncation=True,
            padding=True,
            top_k=None,
        )
        # Define how to convert the text columns to a single string
        if len(di.text_cols) == 1:

            def cols_to_str_fn(array):
                return array[0]

        else:

            def cols_to_str_fn(array):
                return " | ".join(
                    [f"{col}: {val}" for col, val in zip(di.text_cols, array)]
                )

        # LightGBM requires explicitly marking categorical features
        train_df[di.categorical_cols] = train_df[di.categorical_cols].astype("category")
        test_df_sample[di.categorical_cols] = test_df_sample[
            di.categorical_cols
        ].astype("category")

        tab_model = lgb.LGBMClassifier(random_state=42)
        tab_model.fit(train_df[di.tab_cols], y_train)

        if model_type == "ensemble_50":
            model = WeightedEnsemble(
                tab_model=tab_model,
                text_pipeline=text_pipeline,
                text_weight=0.5,
                cols_to_str_fn=cols_to_str_fn,
            )
        elif model_type == "ensemble_75":
            model = WeightedEnsemble(
                tab_model=tab_model,
                text_pipeline=text_pipeline,
                text_weight=0.75,
                cols_to_str_fn=cols_to_str_fn,
            )
        elif model_type == "ensemble_25":
            model = WeightedEnsemble(
                tab_model=tab_model,
                text_pipeline=text_pipeline,
                text_weight=0.25,
                cols_to_str_fn=cols_to_str_fn,
            )
        elif model_type == "stack":
            """
            For the stack model, we make predictions on the validation set. These predictions
            are then used as features for the stack model (another LightGBM model) along with
            the other tabular features. In doing so the stack model learns, depending on the
            tabular features, when to trust the tabular model and when to trust the text model.
            """
            val_df = load_dataset(
                di.ds_name, split="validation", download_mode="force_redownload"
            ).to_pandas()
            val_df[di.categorical_cols] = val_df[di.categorical_cols].astype("category")
            y_val = val_df[di.label_col]
            val_text = list(map(cols_to_str_fn, val_df[di.text_cols].values))

            # Training set is the preditions from the tabular and text models on the validation set
            # plus the tabular features from the validation set
            text_val_preds = text_pipeline(val_text)
            # text_val_preds = np.array(
            #     [format_text_pred(pred) for pred in text_val_preds]
            # )
            text_val_preds = np.array(
                [[lab["score"] for lab in pred] for pred in text_val_preds]
            )

            # add text and tabular predictions to the val_df
            stack_val_df = val_df[di.tab_cols]
            tab_val_preds = tab_model.predict_proba(stack_val_df)
            for i in range(text_val_preds.shape[1]):
                stack_val_df[f"text_pred_{i}"] = text_val_preds[:, i]
            for i in range(tab_val_preds.shape[1]):
                stack_val_df[f"tab_pred_{i}"] = tab_val_preds[:, i]

            stack_model = lgb.LGBMClassifier(random_state=42)
            stack_model.fit(stack_val_df, y_val)

            model = StackModel(
                tab_model=tab_model,
                text_pipeline=text_pipeline,
                stack_model=stack_model,
                cols_to_str_fn=cols_to_str_fn,
            )
        else:
            raise ValueError(f"Invalid model type of {model_type}")

    np.random.seed(1)
    test_sample_vals = test_df_sample[di.tab_cols + di.text_cols].values
    test_vals = test_df[di.tab_cols + di.text_cols].iloc[:10].values
    # train_sample = train_df.sample(1000, random_state=55)
    # train_sample_y = train_sample[di.label_col]
    # train_sample_vals = train_sample[di.tab_cols + di.text_cols].values
    # return model.predict(train_sample_vals), train_sample_y
    return model.predict(test_vals), test_df[di.label_col].iloc[:10].values
    return (
        model.predict(test_sample_vals),
        test_df_sample[di.label_col].values,
        model.predict(test_vals),
        test_df[di.label_col].values,
    )

In [ ]:
def run_model(model_type, ds_type, test_set_size=100, tab_scale_factor=2):
    di = get_dataset_info(ds_type, model_type)
    # Data
    train_df = load_dataset(
        di.ds_name, split="train", download_mode="force_redownload"
    ).to_pandas()
    y_train = train_df[di.label_col]

    test_df = load_dataset(
        di.ds_name, split="test", download_mode="force_redownload"
    ).to_pandas()
    test_df_sample = test_df.sample(test_set_size, random_state=55)

    # Models
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
    if model_type == "all_text":
        text_pipeline = pipeline(
            "text-classification",
            # model=di.text_model_name,
            model="../models/wine/glowing-morning-9/checkpoint-6705",
            tokenizer=tokenizer,
            device="cuda:0",
            truncation=True,
            padding=True,
            top_k=None,
        )
        # Define how to convert all columns to a single string

        def cols_to_str_fn(array):
            return " | ".join(
                [f"{col}: {val}" for col, val in zip(
                    di.tab_cols + di.text_cols, array)]
            )

        model = AllAsTextModel(
            text_pipeline=text_pipeline,
            cols_to_str_fn=cols_to_str_fn,
            # cols=di.tab_cols + di.text_cols
        )
    else:
        text_pipeline = pipeline(
            "text-classification",
            model=di.text_model_name,
            tokenizer=tokenizer,
            device="cuda:0",
            truncation=True,
            padding=True,
            top_k=None,
        )
        # Define how to convert the text columns to a single string
        if len(di.text_cols) == 1:

            def cols_to_str_fn(array):
                return array[0]

        else:

            def cols_to_str_fn(array):
                return " | ".join(
                    [f"{col}: {val}" for col, val in zip(di.text_cols, array)]
                )

        # LightGBM requires explicitly marking categorical features
        train_df[di.categorical_cols] = train_df[di.categorical_cols].astype(
            "category")
        test_df_sample[di.categorical_cols] = test_df_sample[
            di.categorical_cols
        ].astype("category")

        tab_model = lgb.LGBMClassifier(random_state=42)
        tab_model.fit(train_df[di.tab_cols], y_train)

        if model_type == "ensemble_50":
            model = WeightedEnsemble(
                tab_model=tab_model,
                text_pipeline=text_pipeline,
                text_weight=0.5,
                cols_to_str_fn=cols_to_str_fn,
            )
        elif model_type == "ensemble_75":
            model = WeightedEnsemble(
                tab_model=tab_model,
                text_pipeline=text_pipeline,
                text_weight=0.75,
                cols_to_str_fn=cols_to_str_fn,
            )
        elif model_type == "ensemble_25":
            model = WeightedEnsemble(
                tab_model=tab_model,
                text_pipeline=text_pipeline,
                text_weight=0.25,
                cols_to_str_fn=cols_to_str_fn,
            )
        elif model_type == "stack":
            """
            For the stack model, we make predictions on the validation set. These predictions
            are then used as features for the stack model (another LightGBM model) along with
            the other tabular features. In doing so the stack model learns, depending on the
            tabular features, when to trust the tabular model and when to trust the text model.
            """
            val_df = load_dataset(
                di.ds_name, split="validation", download_mode="force_redownload"
            ).to_pandas()
            val_df[di.categorical_cols] = val_df[di.categorical_cols].astype(
                "category")
            y_val = val_df[di.label_col]
            val_text = list(map(cols_to_str_fn, val_df[di.text_cols].values))

            # Training set is the preditions from the tabular and text models on the validation set
            # plus the tabular features from the validation set
            text_val_preds = text_pipeline(val_text)
            # text_val_preds = np.array(
            #     [format_text_pred(pred) for pred in text_val_preds]
            # )
            text_val_preds = np.array(
                [[lab["score"] for lab in pred] for pred in text_val_preds]
            )

            # add text and tabular predictions to the val_df
            stack_val_df = val_df[di.tab_cols]
            tab_val_preds = tab_model.predict_proba(stack_val_df)
            for i in range(text_val_preds.shape[1]):
                stack_val_df[f"text_pred_{i}"] = text_val_preds[:, i]
            for i in range(tab_val_preds.shape[1]):
                stack_val_df[f"tab_pred_{i}"] = tab_val_preds[:, i]

            stack_model = lgb.LGBMClassifier(random_state=42)
            stack_model.fit(stack_val_df, y_val)

            model = StackModel(
                tab_model=tab_model,
                text_pipeline=text_pipeline,
                stack_model=stack_model,
                cols_to_str_fn=cols_to_str_fn,
            )
        else:
            raise ValueError(f"Invalid model type of {model_type}")

    np.random.seed(1)
    test_sample_vals = test_df_sample[di.tab_cols + di.text_cols].values
    test_vals = test_df[di.tab_cols + di.text_cols].values

    return (
        model.predict(test_sample_vals),
        test_df_sample[di.label_col].values,
        model.predict(test_vals),
        test_df[di.label_col].values,
    )

In [32]:
ds_type = "wine"

for model_type in [
    # "all_text",
    "ensemble_25",
    #    "ensemble_50",
    #    "ensemble_75",
    #    "stack"
]:
    # sample_preds, sample_y, preds, y = run_model(
    #     model_type, ds_type=ds_type, tab_scale_factor=1
    # )
    preds, y = run_model(model_type, ds_type=ds_type, tab_scale_factor=1)

    print(f"\n{model_type} on {ds_type}")
    print(f"Test accuaracy: {np.mean(np.argmax(preds, axis=1) == y)}")
    print(
        f"Test accuaracy (sample): {np.mean(np.argmax(sample_preds, axis=1) == sample_y)}"
    )
    # print(f"Test AUC: {roc_auc_score(y, preds[:, 1])}")
    # print(f"Test AUC (sample): {roc_auc_score(sample_y, sample_preds[:, 1])}")
    print(
        f"Test % by label: {[np.mean(y == label) for label in np.unique(y)]}")
    print(
        f"Test % by label (sample): {[np.mean(sample_y == label) for label in np.unique(y)]}"
    )

Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--wine_reviews_ordinal-c036c1f97c7da848/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--wine_reviews_ordinal-c036c1f97c7da848/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
10it [00:00, 77101.18it/s]


ensemble_25 on wine
Test accuaracy: 0.1
Test accuaracy (sample): 0.13
Test % by label: [0.1, 0.1, 0.2, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
Test % by label (sample): [0.12, 0.06, 0.13, 0.03, 0.1, 0.09, 0.01, 0.04, 0.04]


In [6]:
y


array([14, 29,  5, ...,  5,  3, 26])

In [9]:
np.argmax(preds, axis=1)


array([ 9,  9, 23, ..., 23,  2, 15])